# Reproduce Results from Perdikaris with QUEENS

The purpose of this notebook is the reproduction of some of the results presented in [Perdikaris et al. 2017](http://rspa.royalsocietypublishing.org/content/473/2198/20160751). The code published along with the original publication can be found [here](https://github.com/paraklas/NARGP).

## 1D Sinus function

The first example is the following multi-fidelity expression:

The low-fidelity version is defined as:

$f_l(x) = sin(8\pi x)$

And the high fidelity version as:

$f_h(x) = (x- \sqrt(2)f_l^2(x))$

Below, we will build multi-fidelity surrogate models using the nonlinear autoregressive scheme put forth in [[1](http://rspa.royalsocietypublishing.org/content/473/2198/20160751)] and for comparison the original autoregressive scheme proposed by Kennedy and O'Hagan in [[2](https://academic.oup.com/biomet/article/87/1/1/221217)]. The respective surrogate models will be referred to as NARGP and ICM.


### Input File

In [1]:
# Define multi-fidelity model
options = { 
    "output_dir" : "/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output",
    "experiment-name" : "perdikaris_icm_test",
    "database"  :{
      "address"   : "localhost:27017"
    },
    "method": {
      "method_name": "monte_carlo",
      "method_options" :{
        "seed" : 44,
        "num_samples" : 500,
        "model" : "mf_gp_icm_surrogate_model"
      }
    },
    "mf_gp_icm_surrogate_model" : {
      "type" : "datafit_surrogate_model_mf",
      "interface" : "approximation_interface",
      "parameters" : "parameters",
      "subordinate_model" : "truth_model",
      "subordinate_iterator" : "thruth_model_iterator"
    },
    "approximation_interface" : {
      "type" : "approximation_interface_mf",
      "approximation" : "mf_gp_regression"
    },
    "mf_gp_regression" : {
      "type" : "mf_icm_gp_approximation_gpy",
    },
    "mf_gp_nargp_surrogate_model" : {
      "type" : "datafit_surrogate_model_mf",
      "interface" : "approximation_interface2",
      "parameters" : "parameters",
      "subordinate_model" : "truth_model",
      "subordinate_iterator" : "thruth_model_iterator"
    },
    "approximation_interface2" : {
      "type" : "approximation_interface_mf",
      "approximation" : "mf_gp_regression2"
    },
    "mf_gp_regression2" : {
      "type" : "mf_nar_gp_approximation_gpy",
    },
    "thruth_model_iterator" : {
      "method_name": "lhs_mf",
      "method_options" :{
        "seed" : 42,
        "num_samples" : [50, 10],
        "num_iterations" :10,
        "mode" : "nested"
      }
    },
    "truth_model" :{
      "type" : "multi_fidelity_model",
      "model_hierarchy" : ["lofi_sin", "hifi_sin"],
      "eval_cost_per_level" :[1, 1],
      "parameters" : "parameters"
    },
    "hifi_sin" : {
      "type" : "simulation_model",
      "interface" : "interface_hifi"
    },
    "lofi_sin" : {
      "type" : "simulation_model",
      "interface" : "interface_lofi"
    },
    "interface_lofi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/perdikaris_1dsin_lofi.py"
    },
    "interface_hifi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/perdikaris_1dsin_hifi.py"
    },
    "parameters" : {
        "x" : {
            "type" : "FLOAT",
            "size" : 1,
            "min"  : 0,
            "max"  : 1,
            "distribution" : "uniform",
            "distribution_parameter" : [0,1]
        }
    }
}



### Run QUEENS


In [2]:
#from pqueens.iterators.iterator import Iterator
# build iterator
#my_iterator = Iterator.from_config_create_iterator(options)
# perform analysis
#my_iterator.run()


### Build step by step

In [3]:
from pqueens.models.model import Model
import numpy as np

np.random.seed(options["method"]["method_options"]["seed"])
num_samples = options["method"]["method_options"]["num_samples"]


# create surrogate models 
icm_model = Model.from_config_create_model("mf_gp_icm_surrogate_model", options)
nargp_model = Model.from_config_create_model("mf_gp_nargp_surrogate_model", options)

distribution_info = nargp_model.get_parameter_distribution_info()
numparams = len(distribution_info)
samples = np.zeros((num_samples, numparams))

i = 0
for distribution in distribution_info:
    # get appropriate random number generator
    random_number_generator = getattr(np.random, distribution['distribution'])
    # make a copy
    my_args = list(distribution['distribution_parameter'])
    my_args.extend([num_samples])
    samples [:, i] = random_number_generator(*my_args)
    i += 1
        
# update model
icm_model.update_model_from_sample_batch(samples)
nargp_model.update_model_from_sample_batch(samples)

/Applications/anaconda/envs/test/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Compute Results

In [32]:
from pqueens.example_simulator_functions.perdikaris_1dsin_hifi import perdikaris_1dsin_hifi
from pqueens.example_simulator_functions.perdikaris_1dsin_lofi import perdikaris_1dsin_lofi

ref_outputs = perdikaris_1dsin_hifi(samples)
ref_outputs_lofi = perdikaris_1dsin_lofi(samples)

icm_outputs = icm_model.evaluate()
nargp_outputs = nargp_model.evaluate()
nargp_samples_lofi, nargp_samples_hifi = nargp_model.subordinate_iterator.samples
nargp_results_lofi, nargp_results_hifi = nargp_model.subordinate_iterator.outputs


[[ 0.85423776]
 [-0.706258  ]
 [-0.8374317 ]
 [-0.99224704]
 [-0.93692861]
 [ 0.76754718]
 [ 0.65164495]
 [ 0.14937832]
 [ 0.46243824]
 [-0.8412593 ]
 [-0.96362426]
 [-0.66876521]
 [-0.79499166]
 [-0.01877385]
 [ 0.99870924]
 [ 0.95816516]
 [-0.27747565]
 [ 0.53495201]
 [ 0.21301084]
 [-0.50153131]
 [-0.69183844]
 [-0.17204395]
 [ 0.49513128]
 [ 0.80456065]
 [ 0.89428429]
 [ 0.81337001]
 [ 0.54637916]
 [ 0.58113696]
 [-0.66538107]
 [ 0.0157599 ]
 [ 0.18709347]
 [ 0.77819339]
 [ 0.29263871]
 [-0.99477515]
 [ 0.976949  ]
 [ 0.93914885]
 [-0.91237134]
 [ 0.8135745 ]
 [ 0.49955089]
 [ 0.99631717]
 [-0.69214278]
 [ 0.87929953]
 [-0.35098562]
 [-0.13641769]
 [-0.83499815]
 [-0.97607784]
 [ 0.4268583 ]
 [-0.48108564]
 [-0.96758447]
 [-0.95625768]]
[[0.54074696]
 [0.15620227]
 [0.96050701]
 [0.69245781]
 [0.70170695]
 [0.03481537]
 [0.3467599 ]
 [0.2559659 ]
 [0.10587184]
 [0.21022685]
 [0.19826481]
 [0.47085289]
 [0.4115656 ]
 [0.99925297]
 [0.81452183]
 [0.07404968]
 [0.13618722]
 [0.6025412

In [25]:
def sort_ascending(array_1, array_2):
    """ Sort both arrays in increasing order based on array_1"""
    data = np.hstack((array_1,array_2))
    data = data[data[:,0].argsort()]
    return data[:,0], data[:,1]
    

### Plot results

In [35]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go
import numpy as np
init_notebook_mode(connected=True)

x_sorted, y_sorted = sort_ascending(samples,ref_outputs)
# Create a trace
reference_lofi = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'Hi-fi reference'
)

x_sorted, y_sorted = sort_ascending(nargp_samples_hifi,nargp_results_hifi)
trainingplot_hifi = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers',
    name = 'Hi-fi training'
)

x_sorted, y_sorted = sort_ascending(samples,ref_outputs_lofi)
# Create a trace
reference = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'Lo-fi reference'
)

x_sorted, y_sorted = sort_ascending(nargp_samples_lofi,nargp_results_lofi)
trainingplot_lofi = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers',
    name = 'Lo-fi training'
)

x_sorted, y_sorted = sort_ascending(samples,icm_outputs)
icm = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'ICM'
)

x_sorted, y_sorted = sort_ascending(samples,nargp_outputs)
nargp = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'NARGP'
)



layout = dict(title = 'Multi-Fidelity Surrogate Modelling',
              xaxis = dict(title = 'Input'),
              yaxis = dict(title = 'Output'),
              )

fig = Figure(data=[reference,trainingplot_hifi, reference_lofi,trainingplot_lofi, icm, nargp], layout=layout)



iplot(fig)

### print(np.arange(0,2))
